In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.iv.base_iv import IVScenario


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
BinaryIV.generate_data()

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'